In [1]:
import numpy as np

from doublegis_api.api import Api2Gis

api = Api2Gis()
api.load()
api.describe()

Регион: Санкт-Петербург
Количество рубрик: 27
Количество смежных рубрик: 1332
Количество станций метро: 67
Количество зданий: 52559
Количество организаций: 108962
Количество филиалов: 164955


In [2]:
import mysql.connector
from mysql.connector import errorcode

config = {
  'user': 'infotochka_db',
  'password': 'r8BSa81T',
  'host': '185.5.250.80',
  'database': 'infotochka_db',
  'raise_on_warnings': True,
}

def connect():
    try:
        cnx = mysql.connector.connect(**config)
        cursor = cnx.cursor()
        return cnx, cursor
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Something is wrong with your user name or password")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print(err)

In [21]:
categories = []

cnx, cursor = connect()
try:
    cursor.execute('SELECT * FROM `category`')
    for data in cursor:
        categories.append({'id': data[0], 'name': data[1]})
except mysql.connector.Error as err:
    print("Failed creating database: {}".format(err))

In [22]:
# Мэппинг категорий по имени (началу имени)
# get_db_id_by_api_category_id = dict()
# for r in api.rubrics:
#     try:
#         cat = next((rr for rr in categories if r.name.startswith(rr['name'])))
#     except:
#         continue
#     get_db_id_by_api_category_id[r.id] = cat['id']
# get_db_id_by_api_category_id
get_db_id_by_api_category_id = 
{
    5348144816586754: 2,
    5348144816586755: 5,
    5348144816586756: 6,
    5348144816586757: 8,
    5348144816586758: 9,
    5348144816586759: 12,
    5348144816586760: 13,
    5348144816586761: 15,
    5348144816586762: 4,
    5348144816586763: 20,
    5348144816586764: 19,
    5348144816586766: 11,
    5348144816587498: 17,
    5348144816587539: 7,
    5348144816587721: 21,
    5348144816587787: 10,
    5348144816593299: 18,
    5348144816606284: 3,
    5348144816629619: 14,
    5348144816629624: 16,
    5348144816629655: 1
}

{5348144816586754: 2,
 5348144816586755: 5,
 5348144816586756: 6,
 5348144816586757: 8,
 5348144816586758: 9,
 5348144816586759: 12,
 5348144816586760: 13,
 5348144816586761: 15,
 5348144816586762: 4,
 5348144816586763: 20,
 5348144816586764: 19,
 5348144816586766: 11,
 5348144816587498: 17,
 5348144816587539: 7,
 5348144816587721: 21,
 5348144816587787: 10,
 5348144816593299: 18,
 5348144816606284: 3,
 5348144816629619: 14,
 5348144816629624: 16,
 5348144816629655: 1}

In [23]:
stations = []

cnx, cursor = connect()
try:
    cursor.execute('SELECT * FROM `metro`')
    for data in cursor:
        stations.append({'id': data[1], 'name': data[2]})
except mysql.connector.Error as err:
    print("Failed creating database: {}".format(err))

In [24]:
metros_api = {m.name.lower().replace(' ', ''): m for m in api.metro_stations}
metros_api_by_id = {m.id: m for m in api.metro_stations}


updates_stations = []
for s in stations:
    ss = s
    metro = next((m[1] for m in metros_api.items() 
                  if s['name'].lower().replace(' ', '') in m[0]),
                 None)
    if metro is None:
        continue
    
    ss['2gis_id'] = metro.id
    updates_stations.append({'name': metro.name, 'data': ss})
    
updates_stations.append({
    'name': 'Площадь Александра Невского-1',
    'data': {'2gis_id': 5348810536520958, 'id': 41, 'name': 'Площадь Александра Невского'}
})


def get_db_id_by_name(name):
    try:
        return next(s['data']['id'] for s in updates_stations if s['name'] == name)
    except:
        print(name)
        raise
def get_distances_with_sep(filial):
    distances = []
    for s in updates_stations:
        station_2gisid = s['data']['2gis_id']
        d = filial.metro_distances_json[station_2gisid]
        distances.append('{0:.2f}'.format(d / 1000))
    return '|'.join(distances)

In [25]:
filials = api.filials
# filials = np.array([f for f in filials if f.created_at_json['2gis_appear_at'] != ''])

In [26]:
from filtering.categories import get_filter_cats_ids, filter_by_categories
from filtering.stop_words import get_stop_words, filter_by_stop_words

print()
print('Фильтрация филиалов по категориям')
print('Количество филиалов до фильтрации: {0}'.format(len(filials)))
main, sub = get_filter_cats_ids(api.rubrics, api.sub_rubrics)
print('Для фильтрации найдены {0} категорий и {1} подкатегорий'.format(len(main), len(sub)))
filials = filter_by_categories(filials, api.organizations, main, sub, progress='ipython')
print('Количество филиалов после фильтрации: {0}'.format(len(filials)))

print()
print('Фильтрация филиалов по стоп словам')
print('Количество филиалов до фильтрации: {0}'.format(len(filials)))
stop_words = get_stop_words()
print('Для фильтрации найдены стоп слова: {}'.format(stop_words))
filials = filter_by_stop_words(filials, api.organizations, stop_words, progress='ipython')
print('Количество филиалов после фильтрации: {0}'.format(len(filials)))


Фильтрация филиалов по категориям
Количество филиалов до фильтрации: 164955
Для фильтрации найдены 6 категорий и 585 подкатегорий


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.
IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.
IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.
IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.
IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change

Количество филиалов после фильтрации: 106034

Фильтрация филиалов по стоп словам
Количество филиалов до фильтрации: 106034
Для фильтрации найдены стоп слова: ['институт', 'банкомат', 'клиника', 'рынок']


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.
IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.
IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.
IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.
IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change

Количество филиалов после фильтрации: 101205


In [27]:
from export.excel_export import _process_element
from utility.ipython_utility import log_progress
import datetime
import dateutil.parser

table_name = 'base'
cnx, cursor = connect()
cursor.execute('TRUNCATE TABLE `{0}`'.format(table_name))
add_base = ("INSERT INTO {0} "
            "(company, city, address, phone, phone_mob, email, site, cat, depart, "
            "lat, lng, val, val_min, metro, year, month, life, lastmod, category, del, 2gis_id, "
            "created_at, closed_at) "
            "VALUES (%(company)s, %(city)s, %(address)s, %(phone)s, %(phone_mob)s, %(email)s,"
            "%(site)s, %(cat)s, %(depart)s, %(lat)s, %(lng)s, %(val)s, %(val_min)s,"
            "%(metro)s, %(year)s, %(month)s, %(life)s, %(lastmod)s, %(category)s, "
            "%(del)s, %(2gis_id)s, %(created_at)s, %(closed_at)s)".format(table_name))

identifier = 1
db_filials = {}
for i, filial in enumerate(log_progress(filials, name='Филиалы')):
    row = _process_element(api, filial)
    if row is not None:
        if filial.closed_at_json['2gis_removed_at'] != '':
            closed_time = dateutil.parser.parse(
                filial.closed_at_json['2gis_removed_at']).replace(tzinfo=None)
        else:
            closed_time = None
            
        if filial.created_at_json['2gis_appear_at'] != '':
            created_time = dateutil.parser.parse(
                filial.created_at_json['2gis_appear_at']).replace(tzinfo=None)
        else:
            created_time = None
            
        org = next(o for o in api.organizations if o.id == filial.organization_id)

#       Если категории нет, пропускаю точку
        if len(org.main_rubrics['doublegis_rubrics_ids']) == 0:
            continue
        row_dict = {
            'show': 1,
            'company': row[1],
            'city': 1,
            'address': row[2],
            'phone': '',
            'phone_mob': '',
            'email': '',
            'site': '',
            'cat': row[0],
            'depart': '',
            'lat': str(row[3]),
            'lng': str(row[4]),
            'val': get_distances_with_sep(filial),
            'val_min': str(row[8]),
            'metro': get_db_id_by_name(row[5]),
            'year': closed_time.year if closed_time is not None else datetime.datetime.today().year,
            'month': closed_time.month if closed_time is not None else datetime.datetime.today().month,
            'life': row[11] if row[11] is not None else 0,
            'lastmod': datetime.datetime.today(),
            'category': get_db_id_by_api_category_id[org.main_rubrics['doublegis_rubrics_ids'][0]],
            'del': 0,
            '2gis_id': filial.doublegis_id,
            'created_at': created_time,
            'closed_at': closed_time
        }
        db_filials[identifier] = filial
        cursor.execute(add_base, row_dict)
        cnx.commit()
        identifier += 1

cursor.close()
cnx.close()
print('Записано {} филиалов'.format(identifier - 1))

Записано 61625 филиалов


In [10]:
import itertools
from dateutil.relativedelta import relativedelta

def date_generator(month_range):
    from_date = datetime.datetime.today()
    while True:
        yield from_date
        from_date = from_date - relativedelta(months=month_range)
        
dates = list(itertools.islice(date_generator(3), 80))

In [11]:
cnx, cursor = connect()
cursor.execute('TRUNCATE TABLE `base_date_copy`')
add_base = ("INSERT INTO base_date_copy "
            "(base, year, month, lastmod, kon1, kon2, kon3, kon4) "
            "VALUES (%(base)s, %(year)s, %(month)s, %(lastmod)s, "
            "%(kon1)s, %(kon2)s, %(kon3)s, %(kon4)s)")


for date in log_progress(dates, every=1):
    for i, filial in db_filials.items():
        created_at = dateutil.parser.parse(
            filial.created_at_json['2gis_appear_at']).replace(tzinfo=None)
        
        if created_at > date:
#           Филиал открылся позже чем рассматриваемая дата
            continue
    
        if filial.closed_at_json['2gis_removed_at'] != '':
            removed_at = dateutil.parser.parse(
                filial.closed_at_json['2gis_removed_at']).replace(tzinfo=None)
            if removed_at < date:
#               Филиал закрылся раньше, чем рассматриваемая дата
                continue
            
        row_dict = {
            'base': i,
            'year': date.year,
            'month': date.month,
            'lastmod': datetime.datetime.today(),
            'kon1': '',
            'kon2': '',
            'kon3': '',
            'kon4': ''
        }
        
        cursor.execute(add_base, row_dict)
        cnx.commit()
cursor.close()
cnx.close()

Widget Javascript not detected.  It may not be installed or enabled properly.
